In [ ]:
import math
import numpy as np
import re

from collections import Counter
from matplotlib import pyplot as plt
from PIL import Image

PATH = '/home/fizzer/enph353_git/beep-boop/labs/lab5/enph353_cnn_lab/pictures'
# labels_raw = !ls "{PATH}"
# labels = labels_raw[0].split()
# print(labels)

def files_in_folder(folder_path):
  '''
  Returns a list of strings where each entry is a file in the folder_path.
  
  Parameters
  ----------
  
  folder_path : str
     A string to folder for which the file listing is returned.
     
  '''
  files_A = !ls "{folder_path}"
  # The files when listed from Google Drive have a particular format. They are
  # grouped in sets of 4 and have spaces and tabs as delimiters.
  
  # Split the string listing sets of 4 files by tab and space and remove any 
  # empty splits.
  files_B = [list(filter(None, re.split('\t|\s', files))) for files in files_A]
  
  # Concatenate all splits into a single sorted list
  files_C = []
  for element in files_B:
    files_C = files_C + element
  files_C.sort()
  
  return files_C

folder = PATH

files = files_in_folder(folder)

# Crop the images in the folder to get individual letters
for file in files:
    # Open the image
    image = Image.open(folder+"/"+file)

# imgset = np.array([[np.array(Image.open(f'{folder}/{file}')), 0]
#                     for file in files[:]])
# print("Loaded {:} images from folder:\n{}".format(imgset.shape[0], folder))

# Put the image set in X

# 